In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests
import zipfile

: 

## Data Collection

### Files sources

In [94]:
# URLs of the files
train_datas_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

def unzip_file(file_name):
    with zipfile.ZipFile(file_name, mode="r") as archive:
        archive.extractall()
    print(f"Unzipped file {file_name} into current working directory")

# Downloading the files
download_file(test_data_url, 'Neighborhood_Market_data.csv')
download_file(train_datas_url, 'module4_exercise_train.zip')
unzip_file('module4_exercise_train.zip')

Downloaded Neighborhood_Market_data.csv from https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv
Downloaded module4_exercise_train.zip from https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip
Unzipped file module4_exercise_train.zip into current working directory


#### CityMart

In [ ]:
# read "CityMart_data.csv"
df_city_mart = pd.read_csv("CityMart_data.csv", index_col=0, parse_dates=["last_modified"], dtype={"store_name": "category"})
df_city_mart.fillna({"package_volume": -1}, inplace=True)

#### Greenfield_Grocers

In [ ]:
# read "Greenfield_Grocers_data.csv"
df_greenfield_grocers = pd.read_csv("Greenfield_Grocers_data.csv", delimiter="|", header=3, index_col=0, parse_dates=["LAST_MODIFIED"], dtype={"STORE_NAME": "category"})
df_greenfield_grocers.columns = [name.lower() for name in df_greenfield_grocers.columns]
df_greenfield_grocers.drop(["1", "unnamed: 12"], axis=1, inplace=True)
df_greenfield_grocers.dropna(how="all", inplace=True)
df_greenfield_grocers.fillna({"dimension_length": -1}, inplace=True)

#### Outlet_data

In [ ]:

dfs_supersaver_outlet =  pd.read_excel("SuperSaver_Outlet_data.xlsx", sheet_name=None)
df_supersaver_outlet_quantity = dfs_supersaver_outlet["Quantity"]
df_supersaver_outlet_info = dfs_supersaver_outlet["Info"]
df_supersaver_outlet_quantity.set_index("item_code", inplace=True)
column_names = list(df_supersaver_outlet_info.columns)
column_names.pop(0)
column_names.append("to_be_dropped")
df_supersaver_outlet_info.columns = column_names
df_supersaver_outlet_info.drop("to_be_dropped", axis=1, inplace=True)
df_supersaver_outlet_info.columns = [name.replace(" ", "_") for name in df_supersaver_outlet_info.columns]
df_supersaver_outlet_info.set_index("item_code", inplace=True)
df_supersaver_outlet = pd.merge(df_supersaver_outlet_info, df_supersaver_outlet_quantity, left_index=True, right_index=True)

#### HighStreet_Bazaar

In [ ]:
df_highstreet_bazaar = pd.read_json("HighStreet_Bazaar_data.json")
df_highstreet_bazaar.set_index("item_code", inplace=True)
df_highstreet_bazaar["store_name"] = df_highstreet_bazaar["store_name"].astype("category")
df_highstreet_bazaar["last_modified"] = pd.to_datetime(df_highstreet_bazaar["last_modified"], unit="ms")
df_highstreet_bazaar.fillna({
    "days_since_last_purchase": -1
}, inplace=True)

#### Aggregate

In [ ]:
data = pd.concat([df_city_mart, df_greenfield_grocers, df_supersaver_outlet, df_highstreet_bazaar], axis=0)


#### Simple baseline

In [101]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):
    
    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)
    
    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None
    
    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)
    
    # Return the average score
    return np.mean(scores)

In [ ]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### API sources

In [ ]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")
    
    except Exception as e:
        print(f"An error occurred: {e}")

password = get_api("https://www.raphaelcousin.com/api/exercise/auth")["password"]
prices = get_api(f"https://www.raphaelcousin.com/api/exercise/{password}/prices")

In [ ]:
df_prices = pd.DataFrame.from_dict(prices, orient="index", columns=["price"])

#### Aggregate

In [ ]:
data = pd.merge(data, df_prices, left_index=True, right_index=True, how='left')

In [ ]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Scrapping sources

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver (e.g., Chrome)
driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
# driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
driver.get(url)

# Wait for the page to fully load (increase time if needed)
time.sleep(5)

# Get the fully rendered page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
tables = soup.find_all('table')

# Close the Selenium WebDriver
driver.quit()

def parse_row(cols):
    assert len(cols) == 4

    return {
        "item_code": cols[0].text,
        "customer_score": int(cols[1].text),
        "total_reviews": int(cols[2].text),
        "updated_timestamp": int(cols[3].text),
    }

# Scrape the second table (Exercise Data)
course_table = tables[1]
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    exercise_data.append(parse_row(cols))

# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data)
df_exercise.set_index("item_code", inplace=True)


#### Aggregate

In [108]:
data = pd.merge(data, df_exercise, left_index=True, right_index=True, how='left')

In [ ]:
get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

### Generating Submission File

In [ ]:

# read
df_storn =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_storn

In [ ]:
df_storn = pd.merge(df_storn, df_prices, left_index=True, right_index=True, how='left')
df_storn = pd.merge(df_storn, df_exercise, left_index=True, right_index=True, how='left')

In [ ]:
x_loss, x_pred = get_simple_baseline(data, fillna_value=-1, drop_cols=['store_name', 'last_modified'], k_fold=5, scaler="standard", model='linear', metric='mae', target_col='quantity_sold', X_data_test = df_storn)

In [ ]:
if x_loss > 20:
    print(f"To fix the x loss: : {x_loss}")
else:
    submission = pd.DataFrame({
        'item_code': df_storn.index,
        'quantity_sold': x_pred
    })

    submission.to_csv('submission.csv', index=False, sep=',')
    submission.head()